# Respiratory Disease Classification using Unidirectional LSTM
## Multi-class Classification (Healthy, COPD, Asthma)

Pipeline lengkap untuk training model LSTM (non-bidirectional) menggunakan MFCC features dari audio cough dan vowel.

**Classes:**
- **Class 0: Healthy**
- **Class 1: COPD** 
- **Class 2: Asthma**

## Pipeline:
1. Load data dari dataclean_cough dan dataclean_vowel
2. Extract MFCC features real-time dari audio files
3. Split berdasarkan train.csv dan test.csv dengan validation split
4. Training SINGLE unified model dengan attention mechanism
5. Evaluasi dengan metrics: Accuracy, F1, Recall, Precision, Confusion Matrix
6. Prediction pada test set
7. Save hasil ke CSV

**NOTE:** This version uses standard (unidirectional) LSTM instead of bidirectional LSTM.

## 1. Import Libraries

In [30]:
import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import librosa
from pathlib import Path
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, 
    fbeta_score, confusion_matrix, classification_report
)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

Using device: cuda


## 2. Configuration

In [31]:
class Config:
    # Paths
    BASE_PATH = Path('/mnt/ml_storage/Final_Project/SOURCE2')
    COUGH_PATH = BASE_PATH / 'dataclean_cough_1'
    VOWEL_PATH = BASE_PATH / 'dataclean_vowel_1'
    TRAIN_CSV = BASE_PATH / 'train.csv'
    TEST_CSV = BASE_PATH / 'test.csv'
    MODEL_DIR = BASE_PATH / 'models'
    
    # Audio parameters - IMPROVED
    SAMPLE_RATE = 16000
    DURATION = 2.0  # Increased from 1.0 to capture more context
    N_FFT = 2048  # Increased for better frequency resolution
    HOP_LENGTH = 512  # Adjusted for new N_FFT
    WIN_LENGTH = 2048
    N_MFCC = 20  # Increased from 13 for richer features
    N_MELS = 64  # Increased from 40
    
    # Model parameters - UNIDIRECTIONAL LSTM
    INPUT_SIZE = 60  # 20 MFCC + 20 Delta + 20 Delta-Delta
    HIDDEN_SIZE = 128  # Reduced to prevent overfitting
    NUM_LAYERS = 2
    OUTPUT_SIZE = 3  # 3 classes: Healthy(0), COPD(1), Asthma(2)
    DROPOUT = 0.4  # Increased dropout
    BIDIRECTIONAL = False  # *** CHANGED: Using unidirectional LSTM ***
    
    # Training parameters - IMPROVED
    BATCH_SIZE = 16  # Smaller batch for better generalization
    LEARNING_RATE = 0.0005  # Slightly lower for stability
    NUM_EPOCHS = 150
    VAL_SPLIT = 0.2
    PATIENCE = 25  # More patience
    GRADIENT_CLIP = 1.0
    WEIGHT_DECAY = 1e-3  # Stronger regularization
    
    # Data augmentation
    USE_AUGMENTATION = True
    MIXUP_ALPHA = 0.2  # Mixup augmentation strength
    
    # Audio combination mode
    COMBINE_MODE = "concat"  # Options: "concat", "average", "cough_only", "vowel_only"
    
config = Config()

# Create model directory
config.MODEL_DIR.mkdir(exist_ok=True)

print("Configuration (UNIDIRECTIONAL LSTM):")
print(f"  Sample Rate: {config.SAMPLE_RATE} Hz")
print(f"  Duration: {config.DURATION} seconds")
print(f"  N_MFCC: {config.N_MFCC}")
print(f"  Input Features: {config.INPUT_SIZE}")
print(f"  Hidden Size: {config.HIDDEN_SIZE}")
print(f"  Output Classes: {config.OUTPUT_SIZE} (0=Healthy, 1=COPD, 2=Asthma)")
print(f"  Batch Size: {config.BATCH_SIZE}")
print(f"  Learning Rate: {config.LEARNING_RATE}")
print(f"  Epochs: {config.NUM_EPOCHS}")
print(f"  Dropout: {config.DROPOUT}")
print(f"  Weight Decay: {config.WEIGHT_DECAY}")
print(f"  Augmentation: {config.USE_AUGMENTATION}")
print(f"  Combine Mode: {config.COMBINE_MODE}")
print(f"  Bidirectional: {config.BIDIRECTIONAL} *** UNIDIRECTIONAL MODE ***")

Configuration (UNIDIRECTIONAL LSTM):
  Sample Rate: 16000 Hz
  Duration: 2.0 seconds
  N_MFCC: 20
  Input Features: 60
  Hidden Size: 128
  Output Classes: 3 (0=Healthy, 1=COPD, 2=Asthma)
  Batch Size: 16
  Learning Rate: 0.0005
  Epochs: 150
  Dropout: 0.4
  Weight Decay: 0.001
  Augmentation: True
  Combine Mode: concat
  Bidirectional: False *** UNIDIRECTIONAL MODE ***


## 3. Class Imbalance Handling

In [32]:
def compute_class_weights(labels, method='sqrt'):
    """
    Compute class weights for imbalanced dataset
    
    Methods:
    - 'balanced': weight = total_samples / (num_classes * class_count) 
    - 'sqrt': weight = sqrt(max_count / class_count) - softer weighting
    - 'none': equal weights for all classes
    """
    unique_classes, class_counts = np.unique(labels, return_counts=True)
    total_samples = len(labels)
    num_classes = len(unique_classes)
    
    if method == 'balanced':
        # Standard sklearn balanced weighting
        class_weights = total_samples / (num_classes * class_counts)
    elif method == 'sqrt':
        # Softer weighting using square root
        max_count = class_counts.max()
        class_weights = np.sqrt(max_count / class_counts)
    else:
        # No weighting
        class_weights = np.ones(num_classes)
    
    print(f"\nClass Imbalance Analysis (method='{method}'):")
    print(f"{'='*60}")
    print(f"{'Class':<15} {'Count':<10} {'Percentage':<15} {'Weight':<10}")
    print(f"{'-'*60}")
    
    class_names = ['Healthy', 'COPD', 'Asthma']
    for cls, count, weight in zip(unique_classes, class_counts, class_weights):
        percentage = count / total_samples * 100
        print(f"{class_names[int(cls)]:<15} {count:<10} {percentage:<15.2f} {weight:<10.4f}")
    
    print(f"{'='*60}")
    print(f"Weight ratio (max/min): {class_weights.max()/class_weights.min():.2f}x")
    
    return torch.FloatTensor(class_weights)


print("Class imbalance handling function defined!")

Class imbalance handling function defined!


## 4. Dataset Class

In [33]:
class RespiratoryDataset(Dataset):
    """Unified dataset that combines cough and vowel audio features"""
    
    def __init__(self, candidate_ids, labels, config, is_test=False, augment=False):
        """
        Args:
            candidate_ids: List of candidate IDs
            labels: List of labels (0: Healthy, 1: COPD, 2: Asthma) or None for test
            config: Configuration object
            is_test: whether this is test set (no labels)
            augment: whether to apply data augmentation (only for training)
        """
        self.candidate_ids = candidate_ids
        self.labels = labels
        self.config = config
        self.is_test = is_test
        self.augment = augment and not is_test  # Only augment training data
        self.cough_path = config.COUGH_PATH
        self.vowel_path = config.VOWEL_PATH
    
    def __len__(self):
        return len(self.candidate_ids)
    
    def _augment_audio(self, audio):
        """Apply random audio augmentations"""
        if not self.augment or np.random.random() > 0.5:
            return audio
        
        # Choose one augmentation randomly
        aug_type = np.random.choice(['noise', 'shift', 'speed', 'gain'])
        
        if aug_type == 'noise':
            # Add random noise
            noise_factor = np.random.uniform(0.001, 0.005)
            noise = np.random.randn(len(audio)) * noise_factor
            audio = audio + noise
            
        elif aug_type == 'shift':
            # Time shift
            shift_max = int(len(audio) * 0.1)
            shift = np.random.randint(-shift_max, shift_max)
            audio = np.roll(audio, shift)
            
        elif aug_type == 'speed':
            # Speed perturbation (subtle)
            speed_factor = np.random.uniform(0.95, 1.05)
            audio = librosa.effects.time_stretch(audio, rate=speed_factor)
            # Adjust length back
            target_len = int(self.config.SAMPLE_RATE * self.config.DURATION)
            if len(audio) > target_len:
                audio = audio[:target_len]
            else:
                audio = np.pad(audio, (0, target_len - len(audio)), mode='constant')
                
        elif aug_type == 'gain':
            # Random gain
            gain_factor = np.random.uniform(0.8, 1.2)
            audio = audio * gain_factor
        
        return audio
    
    def _load_and_extract_features(self, audio_path):
        """Load audio and extract MFCC features"""
        try:
            audio, sr = librosa.load(audio_path, sr=self.config.SAMPLE_RATE)
            
            # Segment to target duration
            target_samples = int(self.config.SAMPLE_RATE * self.config.DURATION)
            if len(audio) > target_samples:
                audio = audio[:target_samples]
            else:
                audio = np.pad(audio, (0, target_samples - len(audio)), mode='constant')
            
            # Apply augmentation (training only)
            if self.augment:
                audio = self._augment_audio(audio)
            
            # Extract MFCC with config parameters
            mfcc = librosa.feature.mfcc(
                y=audio,
                sr=self.config.SAMPLE_RATE,
                n_mfcc=self.config.N_MFCC,
                n_fft=self.config.N_FFT,
                hop_length=self.config.HOP_LENGTH,
                n_mels=self.config.N_MELS
            )
            
            # Extract delta and delta-delta
            delta = librosa.feature.delta(mfcc)
            delta_delta = librosa.feature.delta(mfcc, order=2)
            
            # Combine features (N_MFCC*3, n_frames)
            features = np.vstack([mfcc, delta, delta_delta])
            
            # Transpose to (n_frames, N_MFCC*3)
            features = features.T
            
            # Replace NaN and Inf with zeros
            features = np.nan_to_num(features, nan=0.0, posinf=0.0, neginf=0.0)
            
            # Clip extreme values
            features = np.clip(features, -1e6, 1e6)
            
            return features
            
        except Exception as e:
            print(f"Error loading {audio_path}: {str(e)}")
            # Return zero features on error - calculate expected frames
            expected_frames = int((self.config.SAMPLE_RATE * self.config.DURATION) / self.config.HOP_LENGTH) + 1
            return np.zeros((expected_frames, self.config.INPUT_SIZE))
    
    def __getitem__(self, idx):
        candidate_id = self.candidate_ids[idx]
        
        # Construct audio paths
        cough_audio_path = self.cough_path / candidate_id / 'cough.wav'
        vowel_audio_path = self.vowel_path / candidate_id / 'vowel.wav'
        
        # Load and extract features
        cough_features = self._load_and_extract_features(cough_audio_path)
        vowel_features = self._load_and_extract_features(vowel_audio_path)
        
        # Combine features based on mode
        if self.config.COMBINE_MODE == "concat":
            # Concatenate along feature dimension
            combined = np.concatenate([cough_features, vowel_features], axis=1)
        elif self.config.COMBINE_MODE == "average":
            # Average the features
            combined = (cough_features + vowel_features) / 2.0
        elif self.config.COMBINE_MODE == "cough_only":
            combined = cough_features
        elif self.config.COMBINE_MODE == "vowel_only":
            combined = vowel_features
        else:
            raise ValueError(f"Unknown combine_mode: {self.config.COMBINE_MODE}")
        
        # Convert to tensor
        audio_tensor = torch.FloatTensor(combined)
        
        if self.is_test:
            return audio_tensor, candidate_id
        else:
            label = torch.LongTensor([self.labels[idx]])[0]
            return audio_tensor, label, candidate_id


# Custom collate function for variable-length sequences
def collate_fn(batch):
    """Collate function to handle variable-length sequences"""
    if len(batch[0]) == 2:
        # Test set (audio, candidate_id)
        audio_features = [item[0] for item in batch]
        candidate_ids = [item[1] for item in batch]
        
        # Pad audio sequences
        audio_padded = nn.utils.rnn.pad_sequence(audio_features, batch_first=True)
        lengths = torch.LongTensor([len(x) for x in audio_features])
        
        return audio_padded, lengths, candidate_ids
    else:
        # Training set (audio, label, candidate_id)
        audio_features = [item[0] for item in batch]
        labels = torch.stack([item[1] for item in batch])
        candidate_ids = [item[2] for item in batch]
        
        # Pad audio sequences
        audio_padded = nn.utils.rnn.pad_sequence(audio_features, batch_first=True)
        lengths = torch.LongTensor([len(x) for x in audio_features])
        
        return audio_padded, lengths, labels, candidate_ids


print("Dataset class defined successfully!")

Dataset class defined successfully!


## 4b. Focal Loss for Imbalanced Classes

In [34]:
class FocalLoss(nn.Module):
    """
    Focal Loss for handling class imbalance
    FL(p_t) = -alpha_t * (1 - p_t)^gamma * log(p_t)
    """
    def __init__(self, alpha=None, gamma=2.0, reduction='mean', label_smoothing=0.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha  # Class weights (optional)
        self.gamma = gamma  # Focusing parameter
        self.reduction = reduction
        self.label_smoothing = label_smoothing
        
    def forward(self, inputs, targets):
        """
        Args:
            inputs: (batch_size, num_classes) - logits
            targets: (batch_size,) - class indices
        """
        ce_loss = nn.functional.cross_entropy(
            inputs, targets, reduction='none', 
            weight=self.alpha, label_smoothing=self.label_smoothing
        )
        p_t = torch.exp(-ce_loss)  # Probability of true class
        focal_loss = (1 - p_t) ** self.gamma * ce_loss
        
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

print("Focal Loss defined successfully!")

Focal Loss defined successfully!


## 5. Unidirectional LSTM Model (Simplified Architecture)

In [35]:
class RespiratoryLSTM(nn.Module):
    """Simplified LSTM-based model for respiratory disease classification"""
    
    def __init__(self, config):
        super(RespiratoryLSTM, self).__init__()
        
        self.config = config
        
        # Calculate actual input size based on combine mode
        if config.COMBINE_MODE == "concat":
            input_size = config.INPUT_SIZE * 2  # 120 features (60 cough + 60 vowel)
        else:
            input_size = config.INPUT_SIZE  # 60 features
        
        # First batch norm and LSTM layer
        self.batch_norm1 = nn.BatchNorm1d(input_size)
        self.lstm1 = nn.LSTM(input_size, config.HIDDEN_SIZE, batch_first=True)
        
        # Second batch norm and LSTM layer
        self.batch_norm2 = nn.BatchNorm1d(config.HIDDEN_SIZE)
        self.lstm2 = nn.LSTM(config.HIDDEN_SIZE, config.HIDDEN_SIZE, batch_first=True)
        
        # Classification layers
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(config.DROPOUT)
        self.fc = nn.Linear(config.HIDDEN_SIZE, config.OUTPUT_SIZE)
    
    def forward(self, x, lengths=None):
        """
        Args:
            x: (batch_size, seq_len, feature_dim)
            lengths: (batch_size,) actual lengths of sequences (optional, not used in this simplified version)
        """
        # First LSTM block
        x = x.transpose(1, 2)
        x = self.batch_norm1(x)
        x = x.transpose(1, 2)
        x, _ = self.lstm1(x)
        
        # Second LSTM block
        x = x.transpose(1, 2)
        x = self.batch_norm2(x)
        x = x.transpose(1, 2)
        x, _ = self.lstm2(x)
        
        # Take last time step output
        x = x[:, -1, :]
        x = self.dropout(x)
        x = self.fc(x)
        
        return x
    
    def _init_weights(self):
        """Initialize weights to avoid initial bias toward any class"""
        for name, param in self.named_parameters():
            if 'weight' in name:
                if 'lstm' in name:
                    # LSTM weights - orthogonal initialization
                    nn.init.orthogonal_(param)
                elif 'batch_norm' in name:
                    # BatchNorm weights
                    nn.init.ones_(param)
                elif len(param.shape) >= 2:
                    # Linear layers - Xavier initialization
                    nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                # All biases to zero
                nn.init.zeros_(param)
        
        # Special: Initialize final classifier layer with small weights
        # to start with near-uniform predictions
        nn.init.xavier_uniform_(self.fc.weight, gain=0.1)
        nn.init.zeros_(self.fc.bias)


def load_checkpoint(checkpoint_path, model):
    """Load model checkpoint with flexible state dict matching"""
    checkpoint_dict = torch.load(checkpoint_path, weights_only=True, map_location='cpu')
    saved_state_dict = checkpoint_dict['model']
    state_dict = model.state_dict()
    new_state_dict = {}
    
    for k, v in state_dict.items():
        try:
            new_state_dict[k] = saved_state_dict[k]
        except:
            new_state_dict[k] = v
    
    model.load_state_dict(new_state_dict)
    return model


# Initialize model
model = RespiratoryLSTM(config).to(device)

# Apply custom weight initialization to avoid class bias
model._init_weights()
print("✓ Applied balanced weight initialization")

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("\nModel Architecture (Simplified Unidirectional LSTM):")
print(model)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print("\nModel uses 2-layer LSTM without attention mechanism")
print("Takes last timestep output for classification")

✓ Applied balanced weight initialization

Model Architecture (Simplified Unidirectional LSTM):
RespiratoryLSTM(
  (batch_norm1): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm1): LSTM(120, 128, batch_first=True)
  (batch_norm2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm2): LSTM(128, 128, batch_first=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (dropout): Dropout(p=0.4, inplace=False)
  (fc): Linear(in_features=128, out_features=3, bias=True)
)

Total parameters: 260,979
Trainable parameters: 260,979

Model uses 2-layer LSTM without attention mechanism
Takes last timestep output for classification


## 6. Training Functions

In [36]:
def mixup_data(x, y, alpha=0.2):
    """Apply Mixup augmentation"""
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    
    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(x.device)
    
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    """Compute Mixup loss"""
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)


def train_epoch(model, dataloader, criterion, optimizer, device, config, use_mixup=False):
    """Train for one epoch"""
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    progress_bar = tqdm(dataloader, desc="Training")
    for audio, lengths, labels, _ in progress_bar:
        audio = audio.to(device)
        labels = labels.to(device)
        
        # Check for NaN in input data
        if torch.isnan(audio).any():
            print("Warning: NaN detected in input data, skipping batch")
            continue
        
        # Apply Mixup augmentation
        if use_mixup and config.MIXUP_ALPHA > 0:
            audio, labels_a, labels_b, lam = mixup_data(audio, labels, config.MIXUP_ALPHA)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(audio, lengths)
        
        # Check for NaN in outputs
        if torch.isnan(outputs).any():
            print("Warning: NaN in model outputs, skipping batch")
            continue
        
        # Compute loss (with or without mixup)
        if use_mixup and config.MIXUP_ALPHA > 0:
            loss = mixup_criterion(criterion, outputs, labels_a, labels_b, lam)
            # For tracking, use original labels
            all_labels.extend(labels_a.cpu().numpy())
        else:
            loss = criterion(outputs, labels)
            all_labels.extend(labels.cpu().numpy())
        
        # Check for NaN in loss
        if torch.isnan(loss):
            print("Warning: NaN loss detected, skipping batch")
            continue
        
        # Backward pass
        loss.backward()
        
        # Gradient clipping to prevent explosion
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=config.GRADIENT_CLIP)
        
        optimizer.step()
        
        # Statistics
        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        
        # Update progress bar
        progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    avg_loss = total_loss / len(dataloader) if len(dataloader) > 0 else float('inf')
    accuracy = accuracy_score(all_labels, all_preds) if len(all_labels) > 0 else 0.0
    
    # Check prediction distribution
    unique_preds, pred_counts = np.unique(all_preds, return_counts=True)
    pred_dist = {int(cls): int(cnt) for cls, cnt in zip(unique_preds, pred_counts)}
    
    return avg_loss, accuracy, pred_dist


def validate_epoch(model, dataloader, criterion, device):
    """Validate the model"""
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for audio, lengths, labels, _ in tqdm(dataloader, desc="Validation"):
            audio = audio.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(audio, lengths)
            loss = criterion(outputs, labels)
            
            # Statistics
            total_loss += loss.item()
            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(probs, dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    
    avg_loss = total_loss / len(dataloader)
    accuracy = accuracy_score(all_labels, all_preds)
    
    return avg_loss, accuracy, all_preds, all_labels, all_probs


def evaluate_model(y_true, y_pred, y_probs=None, title="Evaluation"):
    """Comprehensive model evaluation"""
    print(f"\n{'='*60}")
    print(f"{title}")
    print(f"{'='*60}")
    
    # Metrics
    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = fbeta_score(y_true, y_pred, beta=1, average='weighted', zero_division=0)
    f2 = fbeta_score(y_true, y_pred, beta=2, average='weighted', zero_division=0)
    
    print(f"\nMetrics:")
    print(f"  Accuracy:  {acc:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1 Score:  {f1:.4f}")
    print(f"  F2 Score:  {f2:.4f}")
    
    # Classification report
    print(f"\nClassification Report:")
    class_names = ['Healthy', 'COPD', 'Asthma']
    print(classification_report(y_true, y_pred, target_names=class_names, zero_division=0))
    
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'f2_score': f2,
        'confusion_matrix': cm
    }


def plot_confusion_matrix(cm, title="Confusion Matrix"):
    """Plot confusion matrix"""
    plt.figure(figsize=(8, 6))
    class_names = ['Healthy', 'COPD', 'Asthma']
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()


print("Training functions defined!")

Training functions defined!


## 7. Load and Prepare Data

In [37]:
# Load train/test split
train_df = pd.read_csv(config.TRAIN_CSV)
test_df = pd.read_csv(config.TEST_CSV)

print(f"Training data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")

# =============================================================================
# FILTER OUT SAMPLES WITH MISSING AUDIO FILES
# =============================================================================
print("\n" + "="*60)
print("Filtering out samples with missing audio files...")

def check_audio_exists(candidate_id, config):
    """Check if both cough and vowel audio files exist"""
    cough_path = config.COUGH_PATH / candidate_id / 'cough.wav'
    vowel_path = config.VOWEL_PATH / candidate_id / 'vowel.wav'
    return cough_path.exists() and vowel_path.exists()

# Filter training data
original_train_count = len(train_df)
train_df['has_audio'] = train_df['candidateID'].apply(lambda x: check_audio_exists(x, config))
train_df_filtered = train_df[train_df['has_audio']].copy()
removed_train = original_train_count - len(train_df_filtered)

print(f"Training: {original_train_count} -> {len(train_df_filtered)} (removed {removed_train} with missing files)")

# Filter test data (for prediction, we'll handle missing differently)
original_test_count = len(test_df)
test_df['has_audio'] = test_df['candidateID'].apply(lambda x: check_audio_exists(x, config))
test_missing_ids = test_df[~test_df['has_audio']]['candidateID'].tolist()
print(f"Test: {original_test_count} total, {len(test_missing_ids)} with missing files (will use default prediction)")

# Use filtered training data
train_df = train_df_filtered
print("="*60)

# Check class distribution
print(f"\nClass distribution in training data (after filtering):")
print(train_df['disease'].value_counts().sort_index())
print(f"\nClass percentages:")
class_dist = train_df['disease'].value_counts(normalize=True).sort_index() * 100
for cls, pct in class_dist.items():
    class_name = ['Healthy', 'COPD', 'Asthma'][int(cls)]
    print(f"  Class {cls} ({class_name}): {pct:.2f}%")

# Split training data into train and validation
train_ids = train_df['candidateID'].values
train_labels = train_df['disease'].values

train_ids, val_ids, train_labels, val_labels = train_test_split(
    train_ids, train_labels, 
    test_size=config.VAL_SPLIT, 
    random_state=42,
    stratify=train_labels
)

print(f"\nData split:")
print(f"  Training: {len(train_ids)}")
print(f"  Validation: {len(val_ids)}")
print(f"  Test: {len(test_df)}")

print(f"\nTraining set class distribution:")
unique, counts = np.unique(train_labels, return_counts=True)
for cls, cnt in zip(unique, counts):
    class_name = ['Healthy', 'COPD', 'Asthma'][int(cls)]
    print(f"  Class {cls} ({class_name}): {cnt} samples ({cnt/len(train_labels)*100:.1f}%)")

print(f"\nValidation set class distribution:")
unique, counts = np.unique(val_labels, return_counts=True)
for cls, cnt in zip(unique, counts):
    class_name = ['Healthy', 'COPD', 'Asthma'][int(cls)]
    print(f"  Class {cls} ({class_name}): {cnt} samples ({cnt/len(val_labels)*100:.1f}%)")

# Create datasets
print("\n" + "="*60)
print("Creating datasets...")
train_dataset = RespiratoryDataset(train_ids, train_labels, config, is_test=False, augment=config.USE_AUGMENTATION)
val_dataset = RespiratoryDataset(val_ids, val_labels, config, is_test=False, augment=False)  # No augmentation for validation
test_dataset = RespiratoryDataset(test_df['candidateID'].values, None, config, is_test=True, augment=False)

# Create dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_size=config.BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False
)

val_loader = DataLoader(
    val_dataset,
    batch_size=config.BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=config.BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False
)

print(f"\nDataloaders created:")
print(f"  Train batches: {len(train_loader)}")
print(f"  Val batches: {len(val_loader)}")
print(f"  Test batches: {len(test_loader)}")

Training data shape: (546, 11)
Test data shape: (338, 10)

Filtering out samples with missing audio files...
Training: 546 -> 533 (removed 13 with missing files)
Test: 338 total, 2 with missing files (will use default prediction)

Class distribution in training data (after filtering):
disease
0    137
1    238
2    158
Name: count, dtype: int64

Class percentages:
  Class 0 (Healthy): 25.70%
  Class 1 (COPD): 44.65%
  Class 2 (Asthma): 29.64%

Data split:
  Training: 426
  Validation: 107
  Test: 338

Training set class distribution:
  Class 0 (Healthy): 110 samples (25.8%)
  Class 1 (COPD): 190 samples (44.6%)
  Class 2 (Asthma): 126 samples (29.6%)

Validation set class distribution:
  Class 0 (Healthy): 27 samples (25.2%)
  Class 1 (COPD): 48 samples (44.9%)
  Class 2 (Asthma): 32 samples (29.9%)

Creating datasets...

Dataloaders created:
  Train batches: 27
  Val batches: 7
  Test batches: 22


## 8. Train Unified Model

## 7b. Alternative Strategies for Class Imbalance (Beyond Attention)

If the model is collapsing to one class, try these approaches:

### Option 1: Focal Loss (Already Available)
```python
# Use Focal Loss instead of CrossEntropyLoss
criterion = FocalLoss(alpha=class_weights, gamma=2.0, label_smoothing=0.1)
```

### Option 2: Strong Data Augmentation
- Increase augmentation strength
- Apply multiple augmentations per sample
- Use SpecAugment (time/frequency masking)

### Option 3: Class Balancing via Sampling
- Oversample minority classes
- Undersample majority class
- Use WeightedRandomSampler

### Option 4: Two-Stage Training
- First train on balanced subset
- Then fine-tune on full dataset

### Option 5: Ensemble Methods
- Train multiple models with different random seeds
- Combine predictions via voting/averaging

### Option 6: Feature Engineering
- Use different audio features (spectrograms, wavelet)
- Normalize features per-sample or globally
- Add statistical features (mean, std, skewness)

### Option 7: Stronger Regularization
- Increase dropout (0.5-0.6)
- Add L2 regularization
- Use batch normalization more aggressively

### Option 8: Different Loss Functions
- Use Dice Loss
- Use Balanced CrossEntropy
- Combine multiple losses

**Let's implement the most effective ones below:**

In [ ]:
# ============================================================================
# STRATEGY 1: Weighted Random Sampler for Balanced Batches
# ============================================================================
from torch.utils.data.sampler import WeightedRandomSampler

def create_balanced_sampler(labels):
    """Create a sampler that balances classes in each batch"""
    class_counts = np.bincount(labels)
    class_weights = 1.0 / class_counts
    sample_weights = class_weights[labels]
    sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )
    return sampler

# Create balanced sampler for training
train_sampler = create_balanced_sampler(train_labels)

# Recreate train loader with balanced sampler
train_loader_balanced = DataLoader(
    train_dataset,
    batch_size=config.BATCH_SIZE,
    sampler=train_sampler,  # Use sampler instead of shuffle
    collate_fn=collate_fn,
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False
)

print("✓ Created balanced sampler for training")
print(f"  This ensures each class appears equally in batches")


# ============================================================================
# STRATEGY 2: Class-Balanced Focal Loss
# ============================================================================
# Compute class weights for focal loss
class_counts = np.bincount(train_labels)
class_weights_np = 1.0 / class_counts
class_weights = torch.FloatTensor(class_weights_np).to(device)

# Use focal loss with strong class weights
focal_criterion = FocalLoss(
    alpha=class_weights, 
    gamma=3.0,  # Higher gamma focuses more on hard examples
    label_smoothing=0.1
)

print("✓ Focal Loss with gamma=3.0 ready")


# ============================================================================
# STRATEGY 3: Temperature Scaling for Better Calibration
# ============================================================================
class TemperatureScaledModel(nn.Module):
    """Wraps model with temperature scaling for better probability calibration"""
    def __init__(self, model, temperature=1.5):
        super().__init__()
        self.model = model
        self.temperature = temperature
    
    def forward(self, x, lengths=None):
        logits = self.model(x, lengths)
        return logits / self.temperature

# Optional: wrap model with temperature scaling
# model_scaled = TemperatureScaledModel(model, temperature=2.0).to(device)

print("✓ Temperature scaling available (optional)")


# ============================================================================
# STRATEGY 4: Per-Class Loss Monitoring
# ============================================================================
def compute_per_class_loss(outputs, labels, num_classes=3):
    """Compute loss for each class separately"""
    criterion_per_class = nn.CrossEntropyLoss(reduction='none')
    losses = criterion_per_class(outputs, labels)
    
    per_class_losses = {}
    for c in range(num_classes):
        mask = labels == c
        if mask.sum() > 0:
            per_class_losses[c] = losses[mask].mean().item()
        else:
            per_class_losses[c] = 0.0
    
    return per_class_losses

print("✓ Per-class loss monitoring function ready")


# ============================================================================
# STRATEGY 5: Gradient Surgery (Prevent Dominant Class Gradients)
# ============================================================================
def balance_gradients_by_class(model, outputs, labels, num_classes=3):
    """Scale gradients inversely proportional to class frequency"""
    class_counts = torch.bincount(labels, minlength=num_classes).float()
    total = class_counts.sum()
    class_weights = total / (num_classes * class_counts + 1e-8)
    
    # Weight samples by their class
    sample_weights = class_weights[labels]
    weighted_outputs = outputs * sample_weights.unsqueeze(1)
    
    return weighted_outputs

print("✓ Gradient balancing function ready")


# ============================================================================
# CONFIGURATION: Choose which strategies to use
# ============================================================================
USE_BALANCED_SAMPLER = True
USE_FOCAL_LOSS = True
USE_TEMPERATURE_SCALING = False
USE_GRADIENT_BALANCING = False

print(f"\n{'='*60}")
print("SELECTED STRATEGIES:")
print(f"  Balanced Sampler: {USE_BALANCED_SAMPLER}")
print(f"  Focal Loss: {USE_FOCAL_LOSS}")
print(f"  Temperature Scaling: {USE_TEMPERATURE_SCALING}")
print(f"  Gradient Balancing: {USE_GRADIENT_BALANCING}")
print(f"{'='*60}")

✓ Created balanced sampler for training
  This ensures each class appears equally in batches


NameError: name 'class_weights' is not defined

## 7c. Quick Configuration Changes to Try

If model still collapses, try these **configuration changes** (edit cell 4 above):

### 1. **Increase Dropout** (currently 0.4)
```python
DROPOUT = 0.6  # Much stronger regularization
```

### 2. **Reduce Learning Rate** (currently 0.0005)
```python
LEARNING_RATE = 0.0001  # Slower, more careful learning
```

### 3. **Disable Mixup** (might be confusing the model)
```python
USE_AUGMENTATION = False  # Train without augmentation first
MIXUP_ALPHA = 0.0
```

### 4. **Increase Batch Size** (currently 16)
```python
BATCH_SIZE = 32  # More stable gradients
```

### 5. **Change Combine Mode** (currently "concat")
```python
COMBINE_MODE = "average"  # Simpler feature combination
# or try:
COMBINE_MODE = "cough_only"  # Use only cough data
```

### 6. **Reduce Model Complexity** (currently 128)
```python
HIDDEN_SIZE = 64  # Smaller, simpler model
```

### 7. **Stronger Class Weights**
In cell 7b below, change:
```python
class_weights = compute_class_weights(train_labels, method='balanced')  # Instead of 'sqrt'
```

**Recommended First Try:**
1. Set `DROPOUT = 0.6`
2. Set `LEARNING_RATE = 0.0001`
3. Set `USE_AUGMENTATION = False`
4. Use Balanced Sampler + Focal Loss (already in cell below)

In [ ]:
# ============================================================================
# EMERGENCY FIX: If model still collapses, run this to reset everything
# ============================================================================

def emergency_reset_and_retrain():
    """
    Complete reset with most aggressive anti-collapse settings.
    Use this if model keeps predicting only one class.
    """
    print("EMERGENCY RESET - AGGRESSIVE ANTI-COLLAPSE MODE")
    print("="*60)
    
    # 1. Recreate model with stronger regularization
    class AggressiveConfig:
        def __init__(self, base_config):
            for key, val in vars(base_config).items():
                setattr(self, key, val)
            # Override critical parameters
            self.DROPOUT = 0.7  # Very strong dropout
            self.LEARNING_RATE = 0.00005  # Very slow learning
            self.HIDDEN_SIZE = 64  # Simpler model
            self.USE_AUGMENTATION = False  # No augmentation
            self.MIXUP_ALPHA = 0.0
    
    aggressive_config = AggressiveConfig(config)
    
    # 2. Create new model
    new_model = RespiratoryLSTM(aggressive_config).to(device)
    new_model._init_weights()
    
    # 3. Use strongest class weighting
    strong_weights = compute_class_weights(train_labels, method='balanced')
    strong_weights = strong_weights.to(device)
    strong_weights = strong_weights * 2.0  # Double the effect!
    
    # 4. Focal loss with very high gamma
    strong_criterion = FocalLoss(alpha=strong_weights, gamma=5.0, label_smoothing=0.0)
    
    # 5. Conservative optimizer
    conservative_optimizer = optim.AdamW(
        new_model.parameters(), 
        lr=aggressive_config.LEARNING_RATE,
        weight_decay=0.01  # Strong weight decay
    )
    
    print("✓ Created aggressive anti-collapse model")
    print(f"  Dropout: {aggressive_config.DROPOUT}")
    print(f"  Learning Rate: {aggressive_config.LEARNING_RATE}")
    print(f"  Hidden Size: {aggressive_config.HIDDEN_SIZE}")
    print(f"  Focal gamma: 5.0")
    print(f"  Class weights: {strong_weights.cpu().numpy()}")
    
    return new_model, strong_criterion, conservative_optimizer, aggressive_config

# Uncomment below to use emergency mode:
# model, criterion, optimizer, config = emergency_reset_and_retrain()
# Then rerun the training loop

print("Emergency reset function available - uncomment to use if needed")

Emergency reset function available - uncomment to use if needed


## 7d. 🚨 IMMEDIATE FIX - Stop Training and Apply These Changes

**Your model is still collapsing! Here's what to do NOW:**

### **STEP 1: Stop the current training (interrupt the cell)**

### **STEP 2: Go back to Configuration cell (cell 4) and change these:**

```python
# In Config class, change these values:
DROPOUT = 0.7              # Was 0.4 - INCREASE THIS
LEARNING_RATE = 0.00005    # Was 0.0005 - REDUCE BY 10X
USE_AUGMENTATION = False   # Was True - DISABLE THIS
HIDDEN_SIZE = 64           # Was 128 - REDUCE BY HALF
COMBINE_MODE = "average"   # Was "concat" - SIMPLER COMBINATION
```

### **STEP 3: Run the emergency reset below**

This will use the most aggressive settings to prevent collapse.

In [ ]:
# ============================================================================
# 🚨 EMERGENCY SOLUTION - Use this to retrain with proper settings
# ============================================================================

print("="*70)
print("🚨 APPLYING EMERGENCY ANTI-COLLAPSE CONFIGURATION")
print("="*70)

# Create aggressive configuration
class EmergencyConfig:
    # Paths (keep same)
    BASE_PATH = Path('/mnt/ml_storage/Final_Project/SOURCE2')
    COUGH_PATH = BASE_PATH / 'dataclean_cough_1'
    VOWEL_PATH = BASE_PATH / 'dataclean_vowel_1'
    TRAIN_CSV = BASE_PATH / 'train.csv'
    TEST_CSV = BASE_PATH / 'test.csv'
    MODEL_DIR = BASE_PATH / 'models'
    
    # Audio parameters (keep same)
    SAMPLE_RATE = 16000
    DURATION = 2.0
    N_FFT = 2048
    HOP_LENGTH = 512
    WIN_LENGTH = 2048
    N_MFCC = 20
    N_MELS = 64
    
    # Model parameters - AGGRESSIVE ANTI-COLLAPSE
    INPUT_SIZE = 60
    HIDDEN_SIZE = 32  # 🔥 MUCH SMALLER - was 128
    NUM_LAYERS = 2
    OUTPUT_SIZE = 3
    DROPOUT = 0.8  # 🔥 VERY HIGH - was 0.4
    BIDIRECTIONAL = False
    
    # Training parameters - VERY CONSERVATIVE
    BATCH_SIZE = 8  # 🔥 SMALLER - was 16
    LEARNING_RATE = 0.00001  # 🔥 VERY LOW - was 0.0005
    NUM_EPOCHS = 150
    VAL_SPLIT = 0.2
    PATIENCE = 35  # More patience
    GRADIENT_CLIP = 0.5  # 🔥 STRICTER - was 1.0
    WEIGHT_DECAY = 0.01  # 🔥 STRONGER - was 0.001
    
    # Data augmentation - DISABLED
    USE_AUGMENTATION = False  # 🔥 NO MIXUP
    MIXUP_ALPHA = 0.0
    
    # Audio combination - SIMPLER
    COMBINE_MODE = "average"  # 🔥 SIMPLER - was "concat"

# Apply emergency config
emergency_config = EmergencyConfig()
emergency_config.MODEL_DIR.mkdir(exist_ok=True)

print("\n✓ Emergency Configuration Applied:")
print(f"  Hidden Size: {emergency_config.HIDDEN_SIZE} (was 128)")
print(f"  Dropout: {emergency_config.DROPOUT} (was 0.4)")
print(f"  Learning Rate: {emergency_config.LEARNING_RATE} (was 0.0005)")
print(f"  Batch Size: {emergency_config.BATCH_SIZE} (was 16)")
print(f"  Combine Mode: {emergency_config.COMBINE_MODE} (was concat)")
print(f"  Augmentation: {emergency_config.USE_AUGMENTATION} (was True)")

# Recreate datasets with new config
print("\n🔄 Recreating datasets with emergency config...")
emergency_train_dataset = RespiratoryDataset(
    train_ids, train_labels, emergency_config, 
    is_test=False, augment=emergency_config.USE_AUGMENTATION
)
emergency_val_dataset = RespiratoryDataset(
    val_ids, val_labels, emergency_config, 
    is_test=False, augment=False
)
emergency_test_dataset = RespiratoryDataset(
    test_df['candidateID'].values, None, emergency_config, 
    is_test=True, augment=False
)

# Create balanced sampler for emergency config
emergency_train_sampler = create_balanced_sampler(train_labels)

# Create dataloaders with emergency config
emergency_train_loader = DataLoader(
    emergency_train_dataset,
    batch_size=emergency_config.BATCH_SIZE,
    sampler=emergency_train_sampler,
    collate_fn=collate_fn,
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False
)

emergency_val_loader = DataLoader(
    emergency_val_dataset,
    batch_size=emergency_config.BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False
)

emergency_test_loader = DataLoader(
    emergency_test_dataset,
    batch_size=emergency_config.BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False
)

# Create emergency model
print("\n🏗️ Creating emergency model...")
emergency_model = RespiratoryLSTM(emergency_config).to(device)
emergency_model._init_weights()

# Use VERY STRONG class weights
emergency_class_weights = compute_class_weights(train_labels, method='balanced')
emergency_class_weights = emergency_class_weights.to(device)
emergency_class_weights = emergency_class_weights * 3.0  # 🔥 TRIPLE THE WEIGHTS

print(f"\n🔥 Using TRIPLED class weights:")
print(f"  {emergency_class_weights.cpu().numpy()}")

# Focal loss with VERY HIGH gamma
emergency_criterion = FocalLoss(
    alpha=emergency_class_weights, 
    gamma=5.0,  # 🔥 VERY HIGH - was 3.0
    label_smoothing=0.0  # No label smoothing
)

# Very conservative optimizer
emergency_optimizer = optim.AdamW(
    emergency_model.parameters(), 
    lr=emergency_config.LEARNING_RATE,
    weight_decay=emergency_config.WEIGHT_DECAY
)

# Scheduler
emergency_scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
    emergency_optimizer, T_0=15, T_mult=2, eta_min=1e-7
)

# Count parameters
emergency_params = sum(p.numel() for p in emergency_model.parameters())
print(f"\n✓ Emergency model parameters: {emergency_params:,} (much smaller!)")

print("\n" + "="*70)
print(" EMERGENCY SETUP COMPLETE")
print("="*70)
print("\n IMPORTANT: Use these variables in the training loop:")
print("  - model = emergency_model")
print("  - criterion = emergency_criterion")
print("  - optimizer = emergency_optimizer")
print("  - scheduler = emergency_scheduler")
print("  - config = emergency_config")
print("  - active_train_loader = emergency_train_loader")
print("  - val_loader = emergency_val_loader")
print("\nOr run the emergency training loop in the next cell!")

🚨 APPLYING EMERGENCY ANTI-COLLAPSE CONFIGURATION

✓ Emergency Configuration Applied:
  Hidden Size: 32 (was 128)
  Dropout: 0.8 (was 0.4)
  Learning Rate: 1e-05 (was 0.0005)
  Batch Size: 8 (was 16)
  Combine Mode: average (was concat)
  Augmentation: False (was True)

🔄 Recreating datasets with emergency config...

🏗️ Creating emergency model...

Class Imbalance Analysis (method='balanced'):
Class           Count      Percentage      Weight    
------------------------------------------------------------
Healthy         110        25.82           1.2909    
COPD            190        44.60           0.7474    
Asthma          126        29.58           1.1270    
Weight ratio (max/min): 1.73x

🔥 Using TRIPLED class weights:
  [3.8727272 2.2421052 3.3809524]

✓ Emergency model parameters: 20,763 (much smaller!)

 EMERGENCY SETUP COMPLETE

 IMPORTANT: Use these variables in the training loop:
  - model = emergency_model
  - criterion = emergency_criterion
  - optimizer = emergency_optim

In [ ]:

print("\n" + "="*70)
print("EMERGENCY TRAINING MODE - ANTI-COLLAPSE CONFIGURATION")
print("="*70)

# Use emergency variables
model = emergency_model
criterion = emergency_criterion
optimizer = emergency_optimizer
scheduler = emergency_scheduler
config = emergency_config
active_train_loader = emergency_train_loader
val_loader = emergency_val_loader

# Training history
emergency_history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': [],
    'learning_rate': []
}

best_val_loss = float('inf')
best_val_acc = 0
patience_counter = 0
best_f1 = 0

print(f"\n🔥 Starting EMERGENCY training:")
print(f"  Model size: {emergency_params:,} parameters")
print(f"  Dropout: {config.DROPOUT}")
print(f"  Learning rate: {config.LEARNING_RATE}")
print(f"  Batch size: {config.BATCH_SIZE}")
print(f"  Focal gamma: 5.0")
print(f"  Class weights: TRIPLED")
print(f"\nThis should prevent class collapse!\n")

for epoch in range(config.NUM_EPOCHS):
    print(f"\n{'='*70}")
    print(f"Epoch {epoch+1}/{config.NUM_EPOCHS}")
    print(f"{'='*70}")
    
    # Training phase
    train_loss, train_acc, train_pred_dist = train_epoch(
        model, active_train_loader, criterion, optimizer, device, config, 
        use_mixup=config.USE_AUGMENTATION
    )
    
    # Validation phase
    val_loss, val_acc, val_preds, val_labels_epoch, val_probs = validate_epoch(
        model, val_loader, criterion, device
    )
    
    # Calculate F1 score
    val_f1 = fbeta_score(val_labels_epoch, val_preds, beta=1, average='macro', zero_division=0)
    
    # Check validation prediction distribution
    unique_preds, pred_counts = np.unique(val_preds, return_counts=True)
    val_pred_dist = {int(cls): int(cnt) for cls, cnt in zip(unique_preds, pred_counts)}
    
    # Per-class metrics
    val_f1_per_class = fbeta_score(val_labels_epoch, val_preds, beta=1, average=None, zero_division=0)
    
    # Learning rate scheduling
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    
    # Update history
    emergency_history['train_loss'].append(train_loss)
    emergency_history['train_acc'].append(train_acc)
    emergency_history['val_loss'].append(val_loss)
    emergency_history['val_acc'].append(val_acc)
    emergency_history['learning_rate'].append(current_lr)
    
    # Print results
    print(f"\n📊 Results:")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")
    print(f"  Learning Rate: {current_lr:.6f}")
    print(f"  Train Preds: Class 0={train_pred_dist.get(0,0)}, Class 1={train_pred_dist.get(1,0)}, Class 2={train_pred_dist.get(2,0)}")
    print(f"  Val Preds: Class 0={val_pred_dist.get(0,0)}, Class 1={val_pred_dist.get(1,0)}, Class 2={val_pred_dist.get(2,0)}")
    print(f"  Per-class F1: Healthy={val_f1_per_class[0]:.3f}, COPD={val_f1_per_class[1]:.3f}, Asthma={val_f1_per_class[2]:.3f}")
    
    # Check for class collapse
    num_classes_predicted = len(val_pred_dist)
    if num_classes_predicted == 1:
        print(f"  ⚠️  WARNING: Still predicting only class {list(val_pred_dist.keys())[0]}!")
        print(f"  → Model may need even stronger regularization")
    elif num_classes_predicted == 2:
        missing_class = (set([0,1,2]) - set(val_pred_dist.keys())).pop()
        print(f"  ⚠️  WARNING: Ignoring class {missing_class}!")
    else:
        print(f"  ✅ GOOD: Predicting all 3 classes!")
    
    # Save best model
    if val_f1 > best_f1:
        best_f1 = val_f1
        best_val_loss = val_loss
        best_val_acc = val_acc
        patience_counter = 0
        
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
            'val_acc': val_acc,
            'val_f1': val_f1,
            'config': config,
        }, config.MODEL_DIR / 'emergency_best_model.pth')
        
        print(f"  💾 Best model saved! (F1: {val_f1:.4f}, Acc: {val_acc:.4f})")
    else:
        patience_counter += 1
        print(f"  No improvement ({patience_counter}/{config.PATIENCE})")
    
    # Early stopping
    if patience_counter >= config.PATIENCE:
        print(f"\n⏹️  Early stopping after {epoch+1} epochs")
        break

print("\n" + "="*70)
print("✅ EMERGENCY TRAINING COMPLETE")
print("="*70)
print(f"Best F1: {best_f1:.4f}")
print(f"Best Accuracy: {best_val_acc:.4f}")
print(f"Best Loss: {best_val_loss:.4f}")

# Update history for plotting
history = emergency_history


EMERGENCY TRAINING MODE - ANTI-COLLAPSE CONFIGURATION

🔥 Starting EMERGENCY training:
  Model size: 20,763 parameters
  Dropout: 0.8
  Learning rate: 1e-05
  Batch size: 8
  Focal gamma: 5.0
  Class weights: TRIPLED

This should prevent class collapse!


Epoch 1/150


Validation: 100%|██████████| 14/14 [00:00<00:00, 14.08it/s]



📊 Results:
  Train Loss: 2.9151, Train Acc: 0.3474
  Val Loss: 2.7089, Val Acc: 0.4486, Val F1: 0.2065
  Learning Rate: 0.000010
  Train Preds: Class 0=38, Class 1=313, Class 2=75
  Val Preds: Class 0=0, Class 1=107, Class 2=0
  Per-class F1: Healthy=0.000, COPD=0.619, Asthma=0.000
  ⚠️  WARNING: Still predicting only class 1!
  → Model may need even stronger regularization
  💾 Best model saved! (F1: 0.2065, Acc: 0.4486)

Epoch 2/150


Validation: 100%|██████████| 14/14 [00:01<00:00, 13.93it/s]



📊 Results:
  Train Loss: 2.9965, Train Acc: 0.3474
  Val Loss: 2.7090, Val Acc: 0.4486, Val F1: 0.2065
  Learning Rate: 0.000010
  Train Preds: Class 0=36, Class 1=305, Class 2=85
  Val Preds: Class 0=0, Class 1=107, Class 2=0
  Per-class F1: Healthy=0.000, COPD=0.619, Asthma=0.000
  ⚠️  WARNING: Still predicting only class 1!
  → Model may need even stronger regularization
  No improvement (1/35)

Epoch 3/150


...


📊 Results:
  Train Loss: 2.9249, Train Acc: 0.3239
  Val Loss: 2.7140, Val Acc: 0.2523, Val F1: 0.1343
  Learning Rate: 0.000006
  Train Preds: Class 0=190, Class 1=37, Class 2=199
  Val Preds: Class 0=107, Class 1=0, Class 2=0
  Per-class F1: Healthy=0.403, COPD=0.000, Asthma=0.000
  ⚠️  WARNING: Still predicting only class 0!
  → Model may need even stronger regularization
  No improvement (33/35)

Epoch 71/150


Validation: 100%|██████████| 14/14 [00:01<00:00, 13.00it/s]



📊 Results:
  Train Loss: 2.9285, Train Acc: 0.3122
  Val Loss: 2.7142, Val Acc: 0.2523, Val F1: 0.1343
  Learning Rate: 0.000006
  Train Preds: Class 0=202, Class 1=23, Class 2=201
  Val Preds: Class 0=107, Class 1=0, Class 2=0
  Per-class F1: Healthy=0.403, COPD=0.000, Asthma=0.000
  ⚠️  WARNING: Still predicting only class 0!
  → Model may need even stronger regularization
  No improvement (34/35)

Epoch 72/150


Validation: 100%|██████████| 14/14 [00:01<00:00, 13.39it/s]


📊 Results:
  Train Loss: 2.9575, Train Acc: 0.2981
  Val Loss: 2.7144, Val Acc: 0.2710, Val F1: 0.2127
  Learning Rate: 0.000006
  Train Preds: Class 0=191, Class 1=29, Class 2=206
  Val Preds: Class 0=90, Class 1=0, Class 2=17
  Per-class F1: Healthy=0.393, COPD=0.000, Asthma=0.245
  ⚠️  WARNING: Ignoring class 1!
  No improvement (35/35)

⏹️  Early stopping after 72 epochs

✅ EMERGENCY TRAINING COMPLETE
Best F1: 0.3593
Best Accuracy: 0.3738
Best Loss: 2.7109


In [ ]:
print("\n" + "="*60)
print("TRAINING RESPIRATORY DISEASE CLASSIFICATION MODEL")
print("USING UNIDIRECTIONAL LSTM WITH ANTI-COLLAPSE STRATEGIES")
print("="*60)

# IMPORTANT: Check if we should use emergency config or regular config
# If emergency config was created, we need to use emergency datasets/loaders
try:
    if 'emergency_config' in locals() or 'emergency_config' in globals():
        print("\n⚠️  Emergency config detected - using emergency setup")
        print("If you want to use the regular config, restart the kernel and skip emergency cells")
        config_to_use = emergency_config
        train_loader_to_use = emergency_train_loader
        val_loader_to_use = emergency_val_loader
    else:
        config_to_use = config
        train_loader_to_use = train_loader_balanced if USE_BALANCED_SAMPLER else train_loader
        val_loader_to_use = val_loader
except NameError:
    config_to_use = config
    train_loader_to_use = train_loader_balanced if USE_BALANCED_SAMPLER else train_loader
    val_loader_to_use = val_loader

# 🔧 FIX: Detect actual feature size from data loader
print("\n🔧 Detecting actual feature size from data...")
sample_batch = next(iter(train_loader_to_use))
sample_audio = sample_batch[0]
actual_feature_size = sample_audio.shape[2]
print(f"✓ Data provides {actual_feature_size} features")

# Update config to match actual data
if actual_feature_size == config_to_use.INPUT_SIZE * 2:
    print(f"✓ Data is concatenated (cough + vowel)")
    config_to_use.COMBINE_MODE = "concat"
elif actual_feature_size == config_to_use.INPUT_SIZE:
    print(f"✓ Data is averaged or single-source")
    # Keep existing COMBINE_MODE
else:
    print(f"⚠️ Unexpected feature size: {actual_feature_size}")

# Reinitialize model with proper weights and correct config
print(f"\nReinitializing model with balanced weights...")
print(f"Config being used:")
print(f"  COMBINE_MODE: {config_to_use.COMBINE_MODE}")
print(f"  HIDDEN_SIZE: {config_to_use.HIDDEN_SIZE}")
print(f"  DROPOUT: {config_to_use.DROPOUT}")
print(f"  LEARNING_RATE: {config_to_use.LEARNING_RATE}")
print(f"  Expected input features: {config_to_use.INPUT_SIZE * 2 if config_to_use.COMBINE_MODE == 'concat' else config_to_use.INPUT_SIZE}")

model = RespiratoryLSTM(config_to_use).to(device)
model._init_weights()

# Verify model matches data
print(f"\n✓ Model input layer expects: {model.batch_norm1.num_features} features")
print(f"✓ Data provides: {actual_feature_size} features")
if model.batch_norm1.num_features == actual_feature_size:
    print(f"✅ Model and data are aligned!")
else:
    print(f"❌ ERROR: Model expects {model.batch_norm1.num_features} but data has {actual_feature_size}")
    raise RuntimeError(f"Model-data mismatch! Model expects {model.batch_norm1.num_features} features but data has {actual_feature_size}")

# Compute class weights for balanced training
class_weights = compute_class_weights(train_labels, method='sqrt')
class_weights = class_weights.to(device)

# Select loss function based on strategy
if USE_FOCAL_LOSS:
    print("\nUsing Focal Loss (gamma=3.0) with class weights")
    criterion = FocalLoss(alpha=class_weights, gamma=3.0, label_smoothing=0.1)
else:
    print("\nUsing CrossEntropyLoss with sqrt class weights")
    criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.1)

# Use the appropriate loaders
print(f"Using {'Balanced Sampler' if 'train_loader_balanced' in dir() or 'train_loader_balanced' in locals() else 'Standard'} training data")
active_train_loader = train_loader_to_use
val_loader = val_loader_to_use

# Optimizer with learning rate from config
optimizer = optim.AdamW(model.parameters(), lr=config_to_use.LEARNING_RATE, weight_decay=config_to_use.WEIGHT_DECAY)

# Cosine annealing scheduler - starts high and gradually decreases
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=1e-6)

# Update config reference for training loop
config = config_to_use

# Training history
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': [],
    'learning_rate': []
}

best_val_loss = float('inf')
best_val_acc = 0
patience_counter = 0
best_f1 = 0

print(f"\nStarting training for {config.NUM_EPOCHS} epochs...")
print(f"Using learning rate: {config.LEARNING_RATE}")
print(f"\n{'='*70}")
print("✅ All checks passed - ready to train!")
print(f"{'='*70}\n")

for epoch in range(config.NUM_EPOCHS):
    print(f"\n{'='*60}")
    print(f"Epoch {epoch+1}/{config.NUM_EPOCHS}")
    print(f"{'='*60}")
    
    # Training phase (with mixup if enabled) - USE BALANCED LOADER
    train_loss, train_acc, train_pred_dist = train_epoch(
        model, active_train_loader, criterion, optimizer, device, config, 
        use_mixup=config.USE_AUGMENTATION
    )
    
    # Validation phase
    val_loss, val_acc, val_preds, val_labels_epoch, val_probs = validate_epoch(model, val_loader, criterion, device)
    
    # Calculate F1 score for this epoch
    val_f1 = fbeta_score(val_labels_epoch, val_preds, beta=1, average='macro', zero_division=0)
    
    # Check validation prediction distribution
    unique_preds, pred_counts = np.unique(val_preds, return_counts=True)
    val_pred_dist = {int(cls): int(cnt) for cls, cnt in zip(unique_preds, pred_counts)}
    
    # Per-class metrics for monitoring
    val_f1_per_class = fbeta_score(val_labels_epoch, val_preds, beta=1, average=None, zero_division=0)
    
    # Learning rate scheduling (cosine annealing steps each epoch)
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    
    # Update history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['learning_rate'].append(current_lr)
    
    # Print epoch results
    print(f"\nResults:")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")
    print(f"  Learning Rate: {current_lr:.6f}")
    print(f"  Train Predictions: Class 0={train_pred_dist.get(0,0)}, Class 1={train_pred_dist.get(1,0)}, Class 2={train_pred_dist.get(2,0)}")
    print(f"  Val Predictions: Class 0={val_pred_dist.get(0,0)}, Class 1={val_pred_dist.get(1,0)}, Class 2={val_pred_dist.get(2,0)}")
    print(f"  Per-class F1: Healthy={val_f1_per_class[0]:.3f}, COPD={val_f1_per_class[1]:.3f}, Asthma={val_f1_per_class[2]:.3f}")
    
    # Check for class collapse (if model predicts only one class)
    if len(val_pred_dist) == 1:
        print(f"  ⚠️  WARNING: Model is only predicting class {list(val_pred_dist.keys())[0]}!")
        print(f"  Suggestion: Increase class weights or use stronger augmentation")
    elif len(val_pred_dist) == 2:
        print(f"  ⚠️  WARNING: Model is ignoring class {set([0,1,2]) - set(val_pred_dist.keys())}!")
    
    # Save best model based on F1 score (more robust than accuracy for imbalanced data)
    if val_f1 > best_f1:
        best_f1 = val_f1
        best_val_loss = val_loss
        best_val_acc = val_acc
        patience_counter = 0
        
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
            'val_acc': val_acc,
            'val_f1': val_f1,
            'config': config,
        }, config.MODEL_DIR / 'best_unidirectional_lstm_model.pth')
        
        print(f"  ✓ Best model saved! (Val F1: {val_f1:.4f}, Val Acc: {val_acc:.4f})")
    else:
        patience_counter += 1
        print(f"  No improvement ({patience_counter}/{config.PATIENCE})")
    
    # Early stopping
    if patience_counter >= config.PATIENCE:
        print(f"\nEarly stopping triggered after {epoch+1} epochs")
        break

print("\n" + "="*60)
print("TRAINING COMPLETE")
print("="*60)
print(f"Best Validation F1: {best_f1:.4f}")
print(f"Best Validation Accuracy: {best_val_acc:.4f}")
print(f"Best Validation Loss: {best_val_loss:.4f}")


TRAINING RESPIRATORY DISEASE CLASSIFICATION MODEL
USING UNIDIRECTIONAL LSTM WITH ANTI-COLLAPSE STRATEGIES


NameError: name 'config' is not defined

In [ ]:
# =============================================================================
# FIX: Align model input size with actual data dimensions
# =============================================================================
print("\n" + "="*70)
print("? FIX: Aligning Model with Data Dimensions")
print("="*70)

# Get actual feature count from data
sample_batch = next(iter(active_train_loader))
sample_audio, sample_lengths, sample_labels, sample_ids = sample_batch

actual_features = sample_audio.shape[2]
print(f"\n✓ Detected actual feature count from data: {actual_features}")

# Check what the config says
if config.COMBINE_MODE == "concat":
    expected_features = config.INPUT_SIZE * 2
elif config.COMBINE_MODE == "average":
    expected_features = config.INPUT_SIZE
elif config.COMBINE_MODE in ["cough_only", "vowel_only"]:
    expected_features = config.INPUT_SIZE
else:
    expected_features = config.INPUT_SIZE

print(f"✓ Config says COMBINE_MODE='{config.COMBINE_MODE}'")
print(f"✓ Config expected features: {expected_features}")

if actual_features != expected_features:
    print(f"\n⚠️  Mismatch detected! Data has {actual_features} but config expects {expected_features}")
    print(f"💡 Fixing by updating config to match the actual data...")
    
    # Update config to match reality
    if actual_features == config.INPUT_SIZE * 2:
        config.COMBINE_MODE = "concat"
        print(f"   → Changed COMBINE_MODE to 'concat'")
    elif actual_features == config.INPUT_SIZE:
        config.COMBINE_MODE = "average"  # or could be cough_only/vowel_only
        print(f"   → Changed COMBINE_MODE to 'average'")
    
    # Reinitialize model with corrected config
    print(f"\n? Reinitializing model with correct input size...")
    model = RespiratoryLSTM(config).to(device)
    model._init_weights()
    
    # Reinitialize optimizer and scheduler for new model
    optimizer = optim.AdamW(model.parameters(), lr=config.LEARNING_RATE, weight_decay=config.WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=1e-6)
    
    print(f"✅ Model reinitialized successfully!")

# Final verification
print(f"\n✓ Final Verification:")
print(f"  Model input layer expects: {model.batch_norm1.num_features} features")
print(f"  Data provides: {actual_features} features")
print(f"  Match: {'✅ YES' if model.batch_norm1.num_features == actual_features else '❌ NO'}")

print(f"\n✓ Model Configuration:")
print(f"  COMBINE_MODE: {config.COMBINE_MODE}")
print(f"  INPUT_SIZE: {config.INPUT_SIZE}")
print(f"  HIDDEN_SIZE: {config.HIDDEN_SIZE}")
print(f"  DROPOUT: {config.DROPOUT}")
print(f"  LEARNING_RATE: {config.LEARNING_RATE}")

print("\n" + "="*70)
print("✅ Ready to train!")
print("="*70)

## 9. Visualize Training History

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Loss plot
axes[0].plot(history['train_loss'], label='Train Loss', marker='o')
axes[0].plot(history['val_loss'], label='Val Loss', marker='s')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Validation Loss (Unidirectional LSTM)')
axes[0].legend()
axes[0].grid(True)

# Accuracy plot
axes[1].plot(history['train_acc'], label='Train Acc', marker='o')
axes[1].plot(history['val_acc'], label='Val Acc', marker='s')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Training and Validation Accuracy (Unidirectional LSTM)')
axes[1].legend()
axes[1].grid(True)

# Learning rate plot
axes[2].plot(history['learning_rate'], marker='o', color='green')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Learning Rate')
axes[2].set_title('Learning Rate Schedule')
axes[2].set_yscale('log')
axes[2].grid(True)

plt.tight_layout()
plt.show()

## 10. Model Evaluation on Validation Set

In [ ]:
# Load best model
checkpoint = torch.load(config.MODEL_DIR / 'best_unidirectional_lstm_model.pth', map_location=device, weights_only=False)
model.load_state_dict(checkpoint['model_state_dict'])
print(f"Loaded best unidirectional LSTM model from epoch {checkpoint['epoch']+1}")
print(f"  Best Val Loss: {checkpoint['val_loss']:.4f}")
print(f"  Best Val Acc: {checkpoint['val_acc']:.4f}")

# Evaluate on validation set
val_loss, val_acc, val_preds, val_labels, val_probs = validate_epoch(model, val_loader, criterion, device)

# Calculate detailed metrics
val_metrics = evaluate_model(val_labels, val_preds, val_probs, "Validation Set Performance (Unidirectional LSTM)")
plot_confusion_matrix(val_metrics['confusion_matrix'], "Confusion Matrix - Validation Set (Unidirectional LSTM)")

## 11. Prediction on Test Set

In [ ]:
def predict_test_set(model, dataloader, device):
    """Generate predictions for test set"""
    model.eval()
    
    all_preds = []
    all_probs = []
    all_candidate_ids = []
    
    with torch.no_grad():
        for audio, lengths, candidate_ids in tqdm(dataloader, desc="Predicting"):
            audio = audio.to(device)
            
            # Forward pass
            outputs = model(audio, lengths)
            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(probs, dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            all_candidate_ids.extend(candidate_ids)
    
    return all_candidate_ids, all_preds, all_probs


print("\n" + "="*60)
print("GENERATING TEST SET PREDICTIONS (UNIDIRECTIONAL LSTM)")
print("="*60)

# Generate predictions
test_candidate_ids, test_preds, test_probs = predict_test_set(model, test_loader, device)

print(f"\nGenerated predictions for {len(test_preds)} test samples")

# Create predictions dictionary
predictions_dict = {}
for cid, pred, prob in zip(test_candidate_ids, test_preds, test_probs):
    predictions_dict[cid] = {
        'disease': int(pred),
        'probs': prob
    }

# Get all candidates from test_df
all_test_candidates = test_df['candidateID'].tolist()
print(f"Total test candidates in test.csv: {len(all_test_candidates)}")

# Check for missing predictions
missing_candidates = []
for cid in all_test_candidates:
    if cid not in predictions_dict:
        missing_candidates.append(cid)

if missing_candidates:
    print(f"\n⚠️  WARNING: {len(missing_candidates)} candidates missing predictions!")
    print(f"First 5 missing: {missing_candidates[:5]}")
    print(f"Assigning default prediction (class 1 - COPD) to missing candidates...")
    
    # Assign default prediction
    default_class = 1  # COPD
    default_probs = np.array([0.0, 1.0, 0.0])
    
    for cid in missing_candidates:
        predictions_dict[cid] = {
            'disease': default_class,
            'probs': default_probs
        }

# Create submission dataframe with ALL candidates in original order
submission_df = pd.DataFrame({
    'candidateID': all_test_candidates,
    'disease': [predictions_dict[cid]['disease'] for cid in all_test_candidates]
})

# Verify we have the correct number of rows
assert len(submission_df) == len(test_df), f"Submission has {len(submission_df)} rows, expected {len(test_df)}"
print(f"\n✓ Submission has correct number of rows: {len(submission_df)}")

# Save predictions in submission format
submission_path = config.BASE_PATH / 'submission_unidirectional_lstm.csv'
submission_df.to_csv(submission_path, index=False)
print(f"✓ Submission file saved to '{submission_path}'")

# Also save detailed predictions with probabilities
detailed_df = pd.DataFrame({
    'candidateID': all_test_candidates,
    'predicted_disease': [predictions_dict[cid]['disease'] for cid in all_test_candidates],
    'prob_class_0': [predictions_dict[cid]['probs'][0] for cid in all_test_candidates],
    'prob_class_1': [predictions_dict[cid]['probs'][1] for cid in all_test_candidates],
    'prob_class_2': [predictions_dict[cid]['probs'][2] for cid in all_test_candidates]
})
detailed_path = config.BASE_PATH / 'submission_unidirectional_lstm_detailed.csv'
detailed_df.to_csv(detailed_path, index=False)
print(f"✓ Detailed predictions saved to '{detailed_path}'")

# Display first few predictions
print("\nFirst 10 predictions:")
print(submission_df.head(10))

# Prediction distribution
print("\nPrediction distribution:")
pred_counts = submission_df['disease'].value_counts().sort_index()
class_names = ['Healthy', 'COPD', 'Asthma']
for cls, cnt in pred_counts.items():
    print(f"  Class {cls} ({class_names[int(cls)]}): {cnt} ({cnt/len(submission_df)*100:.1f}%)")

if missing_candidates:
    print(f"\nNote: {len(missing_candidates)} candidates received default predictions")

print("\n" + "="*60)
print("PREDICTION COMPLETE")
print("="*60)

## 12. Final Summary

In [ ]:
print("\n" + "="*70)
print("FINAL SUMMARY - UNIDIRECTIONAL LSTM MODEL")
print("="*70)

print("\n1. MODEL ARCHITECTURE")
print(f"   Type: Unidirectional LSTM with Attention Mechanism")
print(f"   Input Size: {config.INPUT_SIZE * 2 if config.COMBINE_MODE == 'concat' else config.INPUT_SIZE}")
print(f"   Hidden Size: {config.HIDDEN_SIZE}")
print(f"   Layers: {config.NUM_LAYERS}")
print(f"   Bidirectional: {config.BIDIRECTIONAL} *** UNIDIRECTIONAL ***")
print(f"   Dropout: {config.DROPOUT}")
print(f"   Total Parameters: {total_params:,}")

print("\n2. TRAINING CONFIGURATION")
print(f"   Combine Mode: {config.COMBINE_MODE}")
print(f"   Batch Size: {config.BATCH_SIZE}")
print(f"   Learning Rate: {config.LEARNING_RATE}")
print(f"   Weight Decay: {config.WEIGHT_DECAY}")
print(f"   Gradient Clip: {config.GRADIENT_CLIP}")
print(f"   Class Weighting: sqrt method")

print("\n3. DATASET")
print(f"   Training: {len(train_dataset)} samples")
print(f"   Validation: {len(val_dataset)} samples")
print(f"   Test: {len(test_dataset)} samples")

print("\n4. PERFORMANCE METRICS (Validation Set)")
print(f"   Accuracy:  {val_metrics['accuracy']:.4f}")
print(f"   Precision: {val_metrics['precision']:.4f}")
print(f"   Recall:    {val_metrics['recall']:.4f}")
print(f"   F1 Score:  {val_metrics['f1_score']:.4f}")
print(f"   F2 Score:  {val_metrics['f2_score']:.4f}")

print("\n5. OUTPUT FILES")
print(f"   Model: {config.MODEL_DIR / 'best_unidirectional_lstm_model.pth'}")
print(f"   Submission: {submission_path}")
print(f"   Detailed: {detailed_path}")

print("\n6. CLASS LABELS")
print(f"   0: Healthy")
print(f"   1: COPD")
print(f"   2: Asthma")

print("\n7. MODEL COMPARISON")
print(f"   This model uses UNIDIRECTIONAL LSTM")
print(f"   - Processes sequences in forward direction only")
print(f"   - Faster training and inference")
print(f"   - Fewer parameters compared to bidirectional")
print(f"   - May be more suitable for time-series data where future context is not needed")

print("\n" + "="*70)
print("PIPELINE COMPLETE!")
print("="*70)